CIFAR-10 이미지 생성하기
=======

데이터 생성용 DCGAN(Deep Convolutional Generative Adversarial Network) 모델구조를 이용해서 CIFAR-10 데이터를 생성하는 모델을 직접 만들어 보기

- 이미지 데이터의 shape가 (32, 32, 3) 입니다. 생성자, 판별자 모델의 입출력 shape 및 모델 구조에 영향이 있습니다.
- 이미지가 RGB 3채널의 컬러이미지로 변경됩니다. 시각화 과정에서 고려할 점이 있습니다.
- 입력데이터 전체 차원이 3~4배 증가하면서, 학습이 진행되는 양상이 다소 달라집니다.
    
    
모델의 구현은 Keras Sequential API를 활용합니다.      
https://www.tensorflow.org/tutorials/generative/dcgan?hl=ko
     
-21.11.16-